In [ ]:
import sympy as sp
from sympy import cos, sin
from sympy import init_printing
from IPython.display import Math, display
init_printing(use_latex=True)

def rotx_sym(th):
    R = sp.Matrix([[1, 0, 0],
                  [0, sp.cos(th), -sp.sin(th)],
                  [0, sp.sin(th), sp.cos(th)]])
    return R
def roty_sym(th):
    R = sp.Matrix([[sp.cos(th), 0, sp.sin(th)],
                   [0, 1, 0],
                   [-sp.sin(th), 0, sp.cos(th)]])
    return R
def rotz_sym(th):
    R = sp.Matrix([[sp.cos(th), -sp.sin(th), 0],
                   [sp.sin(th), sp.cos(th), 0],
                   [0, 0, 1]])
    return R

# making a symbolic SE3 function
def se3_sym(R = sp.eye(3), p = sp.Matrix([[0], [0], [0]])):

    T = sp.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = p
    
    return T

def get_A_sym(dh, jt_type = 'r', q=sp.Symbol('q')):
    if jt_type == 'r':
        A = se3_sym(R = rotz_sym(dh[0]+q)) @\
            se3_sym(p = sp.Matrix([[0], [0], [dh[1]]])) @\
            se3_sym(p = sp.Matrix([[dh[2]], [0], [0]])) @\
            se3_sym(R = rotx_sym(dh[3]))
    elif jt_type == 'p':
        A = se3_sym(R = rotz_sym(dh[0])) @\
            se3_sym(p = sp.Matrix([[0], [0], [dh[1]+q]])) @\
            se3_sym(p = sp.Matrix([[dh[2]], [0], [0]])) @\
            se3_sym(R = rotx_sym(dh[3]))
    else:
        A = None

    return A

In [14]:
# 2 (a) =======================================================================
q1, q2, d1, a2 = sp.symbols('q_1, q_2, d_1, a_2')

dh_param = [[0, d1, 0, sp.pi/2.0],
            [0, 0, a2, 0]]

T_01 = get_A_sym(dh_param[0], 'r', q1)
T_12 = get_A_sym(dh_param[1], 'r', q2)
T_02 = T_01 @ T_12


# Now get the Jacobian
J = sp.zeros(6,2) # 2 joints
for i in range(2):
    Trans = T[i]
    # Get the second to last column and then the first 3 values from that column (rotation in z)
    z0i_1 = Trans[0:3, 2]

    # Get the last column and then the first 3 values from that column (position)
    joint_pos = Trans[0:3, 3]
    Jvi = z0i_1.cross(sp.Matrix([0,0,0])-joint_pos)
    Jwi = z0i_1
    
    J[:,i] = sp.Matrix.vstack(Jvi,Jwi)

print("FINAL J:")
display(J)

⎡cos(q₁)   0   sin(q₁)    0 ⎤
⎢                           ⎥
⎢sin(q₁)   0   -cos(q₁)   0 ⎥
⎢                           ⎥
⎢   0     1.0     0      d₁ ⎥
⎢                           ⎥
⎣   0      0      0      1.0⎦

⎡cos(q₁)⋅cos(q₂)  -sin(q₂)⋅cos(q₁)  sin(q₁)   a₁⋅cos(q₁)⋅cos(q₂)⎤
⎢                                                               ⎥
⎢sin(q₁)⋅cos(q₂)  -sin(q₁)⋅sin(q₂)  -cos(q₁)  a₁⋅sin(q₁)⋅cos(q₂)⎥
⎢                                                               ⎥
⎢    sin(q₂)          cos(q₂)          0       a₁⋅sin(q₂) + d₁  ⎥
⎢                                                               ⎥
⎣       0                0             0             1.0        ⎦

FINAL J:


⎡d₁⋅cos(q₁)          -(-a₁⋅sin(q₂) - d₁)⋅cos(q₁)        ⎤
⎢                                                       ⎥
⎢d₁⋅sin(q₁)          -(-a₁⋅sin(q₂) - d₁)⋅sin(q₁)        ⎥
⎢                                                       ⎥
⎢                    2                     2            ⎥
⎢    0       - a₁⋅sin (q₁)⋅cos(q₂) - a₁⋅cos (q₁)⋅cos(q₂)⎥
⎢                                                       ⎥
⎢ sin(q₁)                      sin(q₁)                  ⎥
⎢                                                       ⎥
⎢ -cos(q₁)                    -cos(q₁)                  ⎥
⎢                                                       ⎥
⎣    0                            0                     ⎦

In [18]:
# 2 (b) ===========================================================
import sys
sys.path.append("C:/Users/danie/Documents/School/MEEN537/HW")
sys.path.append("/home/daniel/Documents/MEEN537/HW/")
import kinematics as kin
from visualization import VizScene
import sympy as sp
import numpy as np
import time
np.set_printoptions(precision=4, suppress=True)

dh = [[0, 0.3, 0, np.pi/2.0],
      [0, 0, 0.3, 0]]

# An example of defining joint types which we may not have done yet. 
# The 4th joint, and 4th row of the DH parameters correspond to a prismatic joint. 
jt_types = ['r', 'r']

# notice how "jt_types" can be passed as an argument into "SerialArm"
arm = kin.SerialArm(dh, jt=jt_types)

# defining two different sets of joint angles
q_set1 = [0, 0]
q_set2 = [np.pi/2, 0]
q_set3 = [np.pi/4, 0]
q_set4 = [0, np.pi/2]

# calculating two different jacobians for the two different joint configurations. 
J1 = arm.jacob(q_set1)
J2 = arm.jacob(q_set2)
J3 = arm.jacob(q_set3)
J4 = arm.jacob(q_set4)

print("from first set of q's, J is:")
print(J1)
print("from second set of q's, J is:")
print(J2)
print("from third set of q's, J is:")
print(J3)


print("When the second link is 90 deg and link_1 is 0, J is:")
print(J4)


from first set of q's, J is:
[[ 0.  -0. ]
 [ 0.3  0. ]
 [ 0.   0.3]
 [ 0.   0. ]
 [ 0.  -1. ]
 [ 1.   0. ]]
from second set of q's, J is:
[[-0.3 -0. ]
 [ 0.   0. ]
 [ 0.   0.3]
 [ 0.   1. ]
 [ 0.  -0. ]
 [ 1.   0. ]]
from third set of q's, J is:
[[-0.2121 -0.    ]
 [ 0.2121  0.    ]
 [ 0.      0.3   ]
 [ 0.      0.7071]
 [ 0.     -0.7071]
 [ 1.      0.    ]]
When the second link is 90 deg and link_1 is 0, J is:
[[-0.  -0.3]
 [ 0.   0. ]
 [ 0.   0. ]
 [ 0.   0. ]
 [ 0.  -1. ]
 [ 1.   0. ]]


In [19]:
# 3 =======================================================================
import sympy as sp
from sympy import cos, sin
from sympy import init_printing
from IPython.display import Math, display
init_printing(use_latex=True)

q1, q2, ac, a1 = sp.symbols('q_1, q_2, a_c, a_1')

Rz_q1 = sp.Matrix([[cos(q1), -sin(q1), 0, 0],
                   [sin(q1), cos(q1),  0, 0],
                   [0,       0,        1, 0],
                   [0,       0,        0, 1]])

Transx_a1 = sp.Matrix([[1, 0, 0, a1],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

Rz_q2 = sp.Matrix([[cos(q2), -sin(q2), 0, 0],
                   [sin(q2), cos(q2),  0, 0],
                   [0,       0,        1, 0],
                   [0,       0,        0, 1]])

Transx_ac = sp.Matrix([[1, 0, 0, ac],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])



T_01 = Rz_q1 @ Transx_a1
T_02 = T_01 @ Rz_q2 @ Transx_ac

T_01 = sp.trigsimp(sp.simplify(T_01.evalf()))
display(T_01)

T_02 = sp.trigsimp(sp.simplify(T_02.evalf()))
display(T_02)

T = [T_01, T_02]

# Now get the Jacobian
J = sp.zeros(6,2) # 2 joints
for i in range(2):
    Trans = T[i]
    # Get the second to last column and then the first 3 values from that column (rotation in z)
    z0i_1 = Trans[0:3, 2]

    # Get the last column and then the first 3 values from that column (position)
    joint_pos = Trans[0:3, 3]
    Jvi = z0i_1.cross(sp.Matrix([0,0,0])-joint_pos)
    Jwi = z0i_1
    
    J[:,i] = sp.Matrix.vstack(Jvi,Jwi)

print("FINAL J:")
display(J)

⎡cos(q₁)  -sin(q₁)   0   a₁⋅cos(q₁)⎤
⎢                                  ⎥
⎢sin(q₁)  cos(q₁)    0   a₁⋅sin(q₁)⎥
⎢                                  ⎥
⎢   0        0      1.0      0     ⎥
⎢                                  ⎥
⎣   0        0       0      1.0    ⎦

⎡cos(q₁ + q₂)  -sin(q₁ + q₂)   0   a₁⋅cos(q₁) + a_c⋅cos(q₁ + q₂)⎤
⎢                                                               ⎥
⎢sin(q₁ + q₂)  cos(q₁ + q₂)    0   a₁⋅sin(q₁) + a_c⋅sin(q₁ + q₂)⎥
⎢                                                               ⎥
⎢     0              0        1.0                0              ⎥
⎢                                                               ⎥
⎣     0              0         0                1.0             ⎦

FINAL J:


⎡1.0⋅a₁⋅sin(q₁)   1.0⋅a₁⋅sin(q₁) + 1.0⋅a_c⋅sin(q₁ + q₂)⎤
⎢                                                      ⎥
⎢-1.0⋅a₁⋅cos(q₁)     -a₁⋅cos(q₁) - a_c⋅cos(q₁ + q₂)    ⎥
⎢                                                      ⎥
⎢       0                           0                  ⎥
⎢                                                      ⎥
⎢       0                           0                  ⎥
⎢                                                      ⎥
⎢       0                           0                  ⎥
⎢                                                      ⎥
⎣      1.0                         1.0                 ⎦

In [26]:
# 4 (b) ===========================================================
import sys
sys.path.append("C:/Users/danie/Documents/School/MEEN537/HW")
sys.path.append("/home/daniel/Documents/MEEN537/HW/")
import kinematics as kin
from visualization import VizScene
import sympy as sp
import numpy as np
import time
import transforms as tr
np.set_printoptions(precision=4, suppress=True)

q = [0,0,.1,0,0,0]

dh = [[q[0], 0, 0, -np.pi/2.0],
      [q[1], 0.154, 0, np.pi/2.0],
      [0, q[2]+0.25, 0, 0],
      [q[3]-np.pi/2, 0, 0, -np.pi/2.0],
      [q[4]-np.pi/2, 0, 0, np.pi/2.0],
      [q[5]+np.pi/2, 0.263, 0, 0]]
 
jt_types = ['r', 'r', 'p', 'r', 'r', 'r']

arm = kin.SerialArm(dh, tip=tr.se3(tr.roty(-np.pi/2)),jt=jt_types)

# calculating two different jacobians for the two different joint configurations. 
J = arm.jacob(q)

print(f"The Jacobian for q={q} is:\n{J}")

# Visualize the arm
viz = VizScene()
viz.add_arm(arm, draw_frames=True)
viz.update(qs=[q])
time_to_run = 30
refresh_rate = 60

for i in range(refresh_rate * time_to_run):
    viz.update()
    time.sleep(1.0/refresh_rate)

viz.close_viz()


The Jacobian for q=[0, 0, 0.1, 0, 0, 0] is:
[[-0.417  0.45   0.    -0.263 -0.    -0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     1.     0.     0.263  0.   ]
 [ 0.     0.     0.     0.     1.     0.   ]
 [ 0.     1.     0.     0.     0.     1.   ]
 [ 1.     0.     0.     1.     0.     0.   ]]
